In [1]:
import json
import os
from collections import Counter
import numpy as np
import progressbar

### Read Data and Splits

In [2]:
data_p = '/root/data_alfred/unlabeled_12k_20201206/seen'
splits_p = '/root/data_alfred/splits/debug_20201210.json'
# splits_p = '/root/data_alfred/splits/unlabeled_12k_20201206.json'

In [3]:
with open(splits_p, 'r') as f:
    splits = json.load(f)

In [4]:
len(splits['augmentation'])

198

In [5]:
# missing an entire subgoal
{'task': 'pick_and_place_simple-AlarmClock-None-Dresser-317/trial_T20200815_024233_857912', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 4}

{'task': 'pick_and_place_simple-AlarmClock-None-Dresser-317/trial_T20200815_024233_857912',
 'repeat_idx': 0,
 'full_traj_success': True,
 'collected_subgoals': 4}

### (Pre Explainer/Baseline Prediction) Filter down to only tasks with subgoal lengths match between actions and extracted object states features

In [21]:
fail = 0
fail_full_success = []
fail_full_fail = []
error_free_split = []

for task in progressbar.progressbar(splits['augmentation']):

    pp_folder = 'pp_model_seq2seq_per_subgoal,name_v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2'
    traj_data_p = os.path.join(data_p, task['task'], pp_folder, f'ann_{task["repeat_idx"]}.json')
    with open(traj_data_p, 'r') as f:
        traj_data = json.load(f)
    metadata_p = os.path.join(data_p, task['task'], pp_folder, 'metadata_states.json')

    extracted_features_p = os.path.join(data_p, task['task'], 'extracted_feature_states.json')
    with open(extracted_features_p, 'r') as f:
        extracted_features = json.load(f)
        
    subgoal_len_features = [(i,len(subgoal)) for i, subgoal in enumerate(extracted_features['instance_visibile'])]
    counter_items = list(Counter([low_a['high_idx'] for low_a in traj_data['plan']['low_actions']]).items())
    counter_items_num = list(Counter(traj_data['num']['low_to_high_idx']).items())
    
    if task['full_traj_success']:
        subgoal_len_features2 = subgoal_len_features[:-1]
        
    try:
        assert counter_items == subgoal_len_features2
        error_free_split.append(task)
#         print('succeed\n', f'counter_items:{counter_items}\n', f'CORRECT subgoal_len_features {subgoal_len_features}\n')
    except:
        if task['full_traj_success']:
            print('failed\n', f'counter_items:{counter_items}\n', f'INCORRECT subgoal_len_features {subgoal_len_features}\n')
            fail_full_success.append(task)
        else:
            fail_full_fail.append(task)
#             import pdb; pdb.set_trace()
#         with open(metadata_p, 'r') as f:
#             metadata = json.load(f)
        fail += 1

 19% (38 of 198) |####                   | Elapsed Time: 0:00:00 ETA:   0:00:00

failed
 counter_items:[(0, 15), (1, 1), (2, 12), (3, 6), (4, 11), (5, 1)]
 INCORRECT subgoal_len_features [(0, 15), (1, 1), (2, 12), (3, 6), (4, 11), (5, 2)]

failed
 counter_items:[(0, 9), (1, 1), (2, 7), (3, 4), (4, 5), (5, 1)]
 INCORRECT subgoal_len_features [(0, 9), (1, 1), (2, 7), (3, 4), (4, 5), (5, 2)]

failed
 counter_items:[(0, 7), (1, 1), (2, 21), (3, 1)]
 INCORRECT subgoal_len_features [(0, 7), (1, 1), (2, 21), (3, 2)]

failed
 counter_items:[(0, 28), (1, 1), (2, 28), (3, 1)]
 INCORRECT subgoal_len_features [(0, 28), (1, 1), (2, 28), (3, 2)]

failed
 counter_items:[(0, 9), (1, 1), (2, 8), (3, 1)]
 INCORRECT subgoal_len_features [(0, 9), (1, 1), (2, 8), (3, 2)]

failed
 counter_items:[(0, 46), (1, 1), (2, 32), (3, 6), (4, 14), (5, 1)]
 INCORRECT subgoal_len_features [(0, 46), (1, 1), (2, 32), (3, 6), (4, 14), (5, 2)]

failed
 counter_items:[(0, 10), (1, 3), (2, 19), (3, 8), (4, 33), (5, 1)]
 INCORRECT subgoal_len_features [(0, 10), (1, 3), (2, 19), (3, 8), (4, 33), (5, 2)]

f

 40% (81 of 198) |#########              | Elapsed Time: 0:00:00 ETA:   0:00:00

failed
 counter_items:[(0, 49), (1, 1), (2, 31), (3, 1)]
 INCORRECT subgoal_len_features [(0, 49), (1, 1), (2, 31), (3, 2)]

failed
 counter_items:[(0, 19), (1, 1), (2, 12), (3, 8), (4, 25), (5, 1)]
 INCORRECT subgoal_len_features [(0, 19), (1, 1), (2, 12), (3, 8), (4, 25), (5, 2)]

failed
 counter_items:[(0, 18), (1, 1), (2, 16), (3, 1)]
 INCORRECT subgoal_len_features [(0, 18), (1, 1), (2, 16), (3, 2)]

failed
 counter_items:[(0, 19), (1, 1), (2, 19), (3, 1)]
 INCORRECT subgoal_len_features [(0, 19), (1, 1), (2, 19), (3, 2)]

failed
 counter_items:[(0, 16), (1, 1), (2, 19), (3, 1)]
 INCORRECT subgoal_len_features [(0, 16), (1, 1), (2, 19), (3, 2)]



 62% (123 of 198) |#############         | Elapsed Time: 0:00:00 ETA:   0:00:00

failed
 counter_items:[(0, 21), (1, 1), (2, 14), (3, 6), (4, 16), (5, 1)]
 INCORRECT subgoal_len_features [(0, 21), (1, 1), (2, 14), (3, 6), (4, 16), (5, 2)]

failed
 counter_items:[(0, 8), (1, 1), (2, 19), (3, 4), (4, 5), (5, 3)]
 INCORRECT subgoal_len_features [(0, 8), (1, 1), (2, 19), (3, 4), (4, 5), (5, 4)]

failed
 counter_items:[(0, 12), (1, 1), (2, 10), (3, 6), (4, 10), (5, 1)]
 INCORRECT subgoal_len_features [(0, 12), (1, 1), (2, 10), (3, 6), (4, 10), (5, 2)]

failed
 counter_items:[(0, 23), (1, 1), (2, 14), (3, 6), (4, 19), (5, 3)]
 INCORRECT subgoal_len_features [(0, 23), (1, 1), (2, 14), (3, 6), (4, 19), (5, 4)]

failed
 counter_items:[(0, 23), (1, 1), (2, 5), (3, 1), (4, 15), (5, 3), (6, 13), (7, 1), (8, 15), (9, 8), (10, 23), (11, 3)]
 INCORRECT subgoal_len_features [(0, 23), (1, 1), (2, 5), (3, 1), (4, 15), (5, 3), (6, 13), (7, 1), (8, 15), (9, 8), (10, 23), (11, 4)]

failed
 counter_items:[(0, 10), (1, 1), (2, 25), (3, 1)]
 INCORRECT subgoal_len_features [(0, 10), (1, 1)

 81% (161 of 198) |#################     | Elapsed Time: 0:00:00 ETA:   0:00:00

failed
 counter_items:[(0, 9), (1, 1), (2, 12), (3, 1), (4, 1), (5, 1), (6, 6), (7, 8), (8, 10), (9, 1)]
 INCORRECT subgoal_len_features [(0, 9), (1, 1), (2, 12), (3, 1), (4, 1), (5, 1), (6, 6), (7, 8), (8, 10), (9, 2)]

failed
 counter_items:[(0, 18), (1, 1), (2, 23), (3, 1)]
 INCORRECT subgoal_len_features [(0, 18), (1, 1), (2, 23), (3, 2)]

failed
 counter_items:[(0, 21), (1, 1), (2, 8), (3, 1), (4, 21), (5, 3), (6, 19), (7, 1), (8, 21), (9, 8), (10, 12), (11, 1)]
 INCORRECT subgoal_len_features [(0, 21), (1, 1), (2, 8), (3, 1), (4, 21), (5, 3), (6, 19), (7, 1), (8, 21), (9, 8), (10, 12), (11, 2)]

failed
 counter_items:[(0, 13), (1, 1), (2, 7), (3, 1)]
 INCORRECT subgoal_len_features [(0, 13), (1, 1), (2, 7), (3, 2)]

failed
 counter_items:[(0, 15), (1, 1), (2, 18), (3, 1)]
 INCORRECT subgoal_len_features [(0, 15), (1, 1), (2, 18), (3, 2)]

failed
 counter_items:[(0, 11), (1, 1), (2, 15), (3, 1)]
 INCORRECT subgoal_len_features [(0, 11), (1, 1), (2, 15), (3, 2)]

failed
 counter_it

100% (198 of 198) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


failed
 counter_items:[(0, 5), (1, 1), (2, 10), (3, 1)]
 INCORRECT subgoal_len_features [(0, 5), (1, 1), (2, 10), (3, 2)]

failed
 counter_items:[(0, 12), (1, 1), (2, 11), (3, 1)]
 INCORRECT subgoal_len_features [(0, 12), (1, 1), (2, 11), (3, 2)]

failed
 counter_items:[(0, 6), (1, 1), (2, 18), (3, 1)]
 INCORRECT subgoal_len_features [(0, 6), (1, 1), (2, 18), (3, 2)]

failed
 counter_items:[(0, 9), (1, 1), (2, 9), (3, 4), (4, 5), (5, 1)]
 INCORRECT subgoal_len_features [(0, 9), (1, 1), (2, 9), (3, 4), (4, 5), (5, 2)]

failed
 counter_items:[(0, 28), (1, 1), (2, 28), (3, 1)]
 INCORRECT subgoal_len_features [(0, 28), (1, 1), (2, 28), (3, 2)]



In [7]:
len(error_free_split)

159

In [8]:
error_free_split[0]

{'task': 'pick_and_place_simple-CellPhone-None-Sofa-224/trial_T20190912_062415_255725',
 'repeat_idx': 0,
 'full_traj_success': True,
 'collected_subgoals': 5}

In [9]:
Counter([t['collected_subgoals'] for t in error_free_split]).most_common()

[(7, 3114),
 (5, 2383),
 (9, 2230),
 (13, 778),
 (8, 729),
 (11, 224),
 (12, 141),
 (14, 62),
 (4, 59),
 (10, 36),
 (6, 31),
 (17, 5),
 (15, 5),
 (20, 2),
 (16, 2)]

In [10]:
Counter([t['full_traj_success'] for t in error_free_split]).most_common()

[(True, 9801)]

### Save the Filtered Splits

In [15]:
# save splits out!
error_free_splits_p = '/root/data_alfred/splits/debug_20201210_filtered.json'

with open(error_free_splits_p, 'w') as f:
    json.dump({'augmentation':error_free_split}, f)

In [16]:
# save splits out!
error_free_splits_p = '/root/data_alfred/splits/debug_20201210_filtered.json'

with open(error_free_splits_p, 'r') as f:
    error_free_split = json.load(f)['augmentation']

In [17]:
len(error_free_split)

159

### Create Toy Split for Testing

In [49]:
# random_indices = np.random.choice(np.arange(len(error_free_split)), size=40)
random_indices = [5598, 4992, 4753, 6209, 7330, 3377, 1974, 3502, 5065,  772, 5522,
       1035, 4816,  732,   85, 3585, 5463, 5424, 5812, 1934, 7250, 1836,
       5260, 3332, 5095, 4063, 5257, 7358,  581, 2470, 1164, 6440,  979,
       6713, 2440,  714, 4139, 5027, 3355, 6355]

In [46]:
Counter([error_free_split[t]['collected_subgoals'] for t in random_indices]).most_common()

[(3, 14), (6, 9), (2, 8), (5, 3), (11, 2), (10, 1), (9, 1), (12, 1), (4, 1)]

In [53]:
Counter([error_free_split[t]['full_traj_success'] for t in random_indices]).most_common()

[(False, 38), (True, 2)]

In [51]:
toy_split = [error_free_split[t] for t in random_indices]
toy_split[0]

{'task': 'pick_cool_then_place_in_recep-Bread-None-Microwave-18/trial_T20200817_101547_073180',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 3}

In [55]:
toy_splits_p = '/root/data_alfred/splits/sample_failed_toy.json'
with open(toy_splits_p, 'w') as f:
    json.dump({'augmentation':toy_split}, f)

In [8]:
toy_splits_p = '/root/data_alfred/splits/sample_failed_toy.json'
with open(toy_splits_p, 'r') as f:
    toy_split = json.load(f)['augmentation']

### (Post Explainer/Baseline Prediction) Truncate predicted instruction subgoals to match true # subgoals

In [41]:
def truncate_extra_subgoals(traj_data, num_subgoals, key='explainer_annotations'):
    assert len(traj_data[key]['anns'][0]['high_descs']) >= num_subgoals
    if len(traj_data[key]['anns'][0]['high_descs']) > num_subgoals:
        traj_data[key]['anns'][0]['high_descs'] = traj_data[key]['anns'][0]['high_descs'][:num_subgoals]
        return True
    else:
        return False

def match_post_prediction_subgoal_lengths(split, data_p, overwrite_traj=False, debug=False):
    '''
    split: a list of tasks {'task':<task name>/<trial id>, 'repeat_idx':int, 'full_traj_success':boolean, 'collected_subgoals':int}
    '''
    adjusts_explainer = 0
    adjusts_baseline = 0
    
    for task in split:
        traj_data_p = os.path.join(data_p, task['task'], 'traj_data.json')
        with open(traj_data_p, 'r') as f:
            traj_data = json.load(f)

        if debug:
            print (task)
            print ('Explainer')
            print (traj_data['explainer_annotations'])
            print ('Baseline')
            print (traj_data['baseline_annotations'])
            print ('----------------------------------------')
            
        true_num_subgoals = len(traj_data['num']['action_high'])-1
        
        # verify that the predicted instructions has # subgoals >= gold # subgoals
        # when predicting in a batch of different tasks, model can decode more than necessary
        assert len(traj_data['explainer_annotations']['anns'][0]['high_descs']) >= true_num_subgoals
        assert len(traj_data['baseline_annotations']['anns'][0]['high_descs']) >= true_num_subgoals
        
        adjusts_explainer += int(truncate_extra_subgoals(traj_data, true_num_subgoals, key='explainer_annotations'))
        adjusts_baseline += int(truncate_extra_subgoals(traj_data, true_num_subgoals, key='baseline_annotations'))

        if debug:
            print (adjusts_explainer, adjusts_baseline)
            print (task)
            print ('Explainer')
            print (traj_data['explainer_annotations'])
            print ('Baseline')
            print (traj_data['baseline_annotations'])
            print ('\n\n\n\n\n\n\n\n')
        
        # be careful about overwriting!
        if overwrite_traj:
            with open(traj_data_p, 'w') as f:
                json.dump(traj_data, f)
    
    return adjusts_explainer, adjusts_baseline
        

In [47]:
# Verify with toy split and toy dataset
print (len(toy_split))
adjusts_explainer, adjusts_baseline = match_post_prediction_subgoal_lengths(toy_split, data_p, overwrite_traj=True, debug=True)

40
{'task': 'pick_cool_then_place_in_recep-Bread-None-Microwave-18/trial_T20200817_101547_073180', 'repeat_idx': 0, 'full_traj_success': False, 'collected_subgoals': 3}
Explainer
{'anns': [{'task_desc': 'put a loaf of bread on a counter .', 'high_descs': ['turn left and walk to the sink then turn left and walk to the wall then turn left and walk to the wall then turn left and walk to the table .', 'pick up a slice of bread from the table .', 'turn around and walk to the fridge .']}]}
Baseline
{'anns': [{'task_desc': 'put a loaf of bread on a counter .', 'high_descs': ['turn left and walk to the table then turn right and walk to the table then turn left and walk to the table then turn left and walk to the fridge then turn left and walk to the fridge then turn left and walk to the table then turn left and walk to the table .', 'pick up the knife on the table .', 'turn left and walk to the refrigerator .']}]}
----------------------------------------
0 0
{'task': 'pick_cool_then_place_in_r

{'task': 'pick_and_place_with_movable_recep-Apple-Pan-SinkBasin-2/trial_T20200814_201204_070571', 'repeat_idx': 0, 'full_traj_success': False, 'collected_subgoals': 6}
Explainer
{'anns': [{'task_desc': 'put a pan with an apple in it on the counter .', 'high_descs': ['turn around and walk to the sink .', 'pick up the apple from the sink .', 'turn left and walk to the stove .', 'put the apple in the pan on the stove .', 'pick up the pan with the apple in it .', 'turn around and walk to the counter to the right of the sink .']}]}
Baseline
{'anns': [{'task_desc': 'put a pan with an apple in it on the counter .', 'high_descs': ['turn around and walk to the sink on the left .', 'pick up a slice of apple from the sink .', 'turn around and walk to the stove to your right .', 'put the apple on the front left corner of the stove .', 'pick up the pan on the stove .', 'turn around , move to the counter , right of the sink .']}]}
----------------------------------------
0 0
{'task': 'pick_and_place

In [46]:
# Full run on explainer/baseline processed traj data
print (len(error_free_split))
match_post_prediction_subgoal_lengths(error_free_split, data_p, overwrite_traj=True, debug=False)

7570


(4838, 4882)

### (Post Explainer/Baseline Prediction) Some Sanity Check, Ignore

In [48]:
error_free_split[0]

{'task': 'look_at_obj_in_light-BaseballBat-None-DeskLamp-301/trial_T20200814_164125_595727',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 2}

In [115]:
error_free_split[6000]

{'task': 'pick_cool_then_place_in_recep-Pot-None-Cabinet-21/trial_T20200818_085037_265666',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 5}

In [65]:
traj_data_p = os.path.join(data_p, 'look_at_obj_in_light-BasketBall-None-DeskLamp-319/trial_T20200814_165040_882757', 'traj_data.json')
with open(traj_data_p, 'r') as f:
    traj_data = json.load(f)

In [116]:
traj_data_p = os.path.join(data_p, 'pick_cool_then_place_in_recep-Pot-None-Cabinet-21/trial_T20200818_085037_265666', 'traj_data.json')
with open(traj_data_p, 'r') as f:
    traj_data = json.load(f)

In [117]:
traj_data['explainer_annotations']

{'anns': [{'task_desc': 'put a cold pan on the table .',
   'high_descs': ['turn to the left and face the white table .',
    'pick up the pot on the stove .',
    'turn around and walk to the fridge .',
    'open the fridge and place the pot on the top shelf , close the door , wait a moment , and take it out again .',
    'turn around and walk to the table .']}]}

In [118]:
traj_data['baseline_annotations']

{'anns': [{'task_desc': 'put a cold pan on the table .',
   'high_descs': ['turn left and walk to the stove .',
    'pick up the pot with the spoon in it .',
    'turn around and go to the white table on the left .',
    'open the fridge , put the pan inside , close the fridge , wait a few seconds , open the fridge , take the pan out , and close the refrigerator .',
    'turn around , go to the white table on the right .']}]}

In [114]:
traj_data['plan']['high_pddl']

[{'discrete_action': {'action': 'GotoLocation', 'args': ['diningtable']},
  'high_idx': 0,
  'planner_action': {'action': 'GotoLocation', 'location': 'loc|-8|-3|3|60'}},
 {'discrete_action': {'action': 'PickupObject', 'args': ['lettuce']},
  'high_idx': 1,
  'planner_action': {'action': 'PickupObject',
   'forceVisible': True,
   'objectId': 'Lettuce|-02.58|+00.80|-00.61',
   'receptacleObjectId': 'DiningTable|-02.87|+00.68|-00.51'}},
 {'discrete_action': {'action': 'GotoLocation', 'args': ['sink']},
  'high_idx': 2,
  'planner_action': {'action': 'GotoLocation', 'location': 'loc|-8|-13|2|60'}},
 {'discrete_action': {'action': 'CleanObject', 'args': ['lettuce']},
  'high_idx': 3,
  'planner_action': {'action': 'CleanObject',
   'cleanObjectId': 'Sink|-01.79|+00.90|-03.75|SinkBasin',
   'forceVisible': True,
   'objectId': 'Sink|-01.79|+00.90|-03.75|SinkBasin',
   'receptacleObjectId': 'Sink|-01.79|+00.90|-03.75|SinkBasin'}},
 {'discrete_action': {'action': 'NoOp', 'args': []},
  'high_